In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import networkx as nx
import json
import time
import sys
sys.path.insert(1, '../../rtsvg')
from rtsvg import *
rt = RACETrack()
df_llama = pd.read_csv('llama3_8b_2014_vast_sbs.csv')
df       = pd.read_csv('phi_small_2014_vast_sbs.csv')
len(df_llama), len(df), df['phi_small_time'].sum()/60.0/60.0 # appears to be about 8 hours...

In [ ]:
_jsons_ = []
_failed_parse_counts_, _failed_parses_, _successful_parse_counts_, _ignored_counts_ = 0, [], 0, 0
for _json_str_ in df['phi_small_response']:
    # Chop to either the first and last curly brace or bracket depending on which occurs first...
    # ... this fixes about half of the parses
    if '{' in _json_str_ and '}' in _json_str_ and '[' in _json_str_ and ']' in _json_str_:
        i0,j0 = _json_str_.index('{'), _json_str_.index('[')
        if i0 < j0: _json_str_ = _json_str_[i0:_json_str_.rindex('}')+1]
        else:       _json_str_ = _json_str_[j0:_json_str_.rindex(']')+1]
    # These next two fix almost all of the other half of the parses...
    elif '{' in _json_str_ and '}' in _json_str_: _json_str_ = _json_str_[_json_str_.index('{'):_json_str_.rindex('}')+1]
    elif '[' in _json_str_ and ']' in _json_str_: _json_str_ = _json_str_[_json_str_.index('['):_json_str_.rindex(']')+1]

    _json_ = None
    try: _json_ = json.loads(_json_str_)
    except:
        try: _json_ = json.loads(_json_str_ + '}')
        except:
            try: _json_ = json.loads(_json_str_ + ']')
            except:
                try: _json_ = json.loads(_json_str_ + '] }')
                except:
                    pass

    if _json_ is None:
        _failed_parse_counts_ += 1
        _failed_parses_.append(_json_str_)
    else:
        _jsons_.append(_json_)
        _successful_parse_counts_ += 1

_successful_parse_counts_ ,_failed_parse_counts_, _ignored_counts_

In [ ]:
#j = 0
#_splits_ = _failed_parses_[j].split('\n')
#for i in range(len(_splits_)):
#    print(f'{i:-3} {_splits_[i]}')
#json.loads(_failed_parses_[j])

In [ ]:
#_set_ = set()
#for i in range(len(_failed_parses_)):
#    _escapes_ = listEscapes(_failed_parses_[i])
#    _set_ = _set_ | set(_escapes_)
#_set_

In [ ]:
_pos_ = {}
from os.path import exists
_json_repr_ = rt.jsonRepr(_jsons_)
_df_, _relates_, _labels_ = _json_repr_.starPathGraphDataFrame()
_df_ = pl.from_pandas(_df_)
_rtg_ = rt.interactiveGraphLayout(_df_, {'relationships':_relates_, 'node_labels':_labels_, 'link_size':'vary', 'pos':_pos_, 'link_opacity':0.2}, w=1900, h=1000)
if exists('phi_small_sbs_layout.csv'): _rtg_.loadLayout('phi_small_sbs_layout.csv')
_rtg_

In [ ]:
#_rtg_.saveLayout('phi_small_sbs_layout.csv')

In [ ]:
_pos_ = _rtg_.pos

In [ ]:
_rtg_.selectEntities('root')